<a href="https://colab.research.google.com/github/AryanJadhao/Natural_Language_Processing/blob/main/distillBert_fintuneing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [11]:
from datasets import load_dataset

dataset = load_dataset("squad")

small_train = dataset["train"].shuffle(seed=42).select(range(5000))
small_val = dataset["validation"].shuffle(seed=42).select(range(1000))



In [12]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)



config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForQuestionAnswering LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
qa_outputs.weight       | MISSING    | 
qa_outputs.bias         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [15]:
max_length = 384
doc_stride = 128

def preprocess(example):

    questions = [q.strip() for q in example["question"]]

    inputs = tokenizer(
        questions,
        example["context"],
        max_length=max_length,
        truncation="only_second",
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = example["answers"][sample_idx]

        if len(answer["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        sequence_ids = inputs.sequence_ids(i)

        # find context token range
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # check if answer inside span
        if not (offsets[context_start][0] <= start_char and
                offsets[context_end][1] >= end_char):
            start_positions.append(0)
            end_positions.append(0)
        else:
            # find start token
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            # find end token
            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


In [16]:
tokenized = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./fast-qa",
    num_train_epochs=1,                # KEY
    per_device_train_batch_size=16,    # bigger batch
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    logging_steps=50,
    save_strategy="no",
    eval_strategy="no",          # skip eval during train
    fp16=True                          # if GPU
)



In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
)

trainer.train()



Step,Training Loss
50,4.701844
100,3.707778
150,3.229884
200,2.559343
250,2.522950
300,2.289266
350,2.065768
400,1.967028
450,1.924124
500,1.890999


TrainOutput(global_step=5533, training_loss=1.4883494409023046, metrics={'train_runtime': 876.4639, 'train_samples_per_second': 101.001, 'train_steps_per_second': 6.313, 'total_flos': 8674451270424576.0, 'train_loss': 1.4883494409023046, 'epoch': 1.0})

In [25]:
trainer.save_model("fast-qa")
tokenizer.save_pretrained("fast-qa")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('fast-qa/tokenizer_config.json', 'fast-qa/tokenizer.json')

In [26]:
import os
print(os.listdir("fast-qa"))


['config.json', 'model.safetensors', 'tokenizer_config.json', 'training_args.bin', 'tokenizer.json']


In [27]:
from transformers import pipeline

qa = pipeline(
    "question-answering",
    model="./fast-qa",          # your saved folder
    tokenizer="./fast-qa"
)


Loading weights:   0%|          | 0/102 [00:00<?, ?it/s]

In [45]:
context = """Artificial Intelligence (AI) is a broad field of computer science focused on building systems that can perform tasks typically requiring human intelligence. These tasks include reasoning, learning, perception, language understanding, and decision-making. The term Artificial Intelligence was first coined in 1956 at the Dartmouth Conference by John McCarthy, who is often called the father of AI. Early AI systems were rule-based and relied heavily on symbolic logic and handcrafted knowledge.
  Machine Learning is a subfield of AI that enables computers to learn patterns from data instead of being explicitly programmed. One of the earliest machine learning algorithms was the perceptron, introduced by Frank Rosenblatt in 1957. Later, in the 1980s and 1990s, statistical learning methods such as decision trees and support vector machines became popular. A major breakthrough came in 2012 when deep learning gained attention after a neural network model significantly outperformed others in the ImageNet competition.
  Deep learning models are based on artificial neural networks with multiple layers. These networks are inspired by the structure of the human brain but are mathematically very different. Geoffrey Hinton, Yoshua Bengio, and Yann LeCun are considered pioneers of deep learning and jointly received the Turing Award in 2018 for their contributions. Deep learning is widely used in computer vision, speech recognition, and natural language processing.

Natural Language Processing (NLP) focuses on enabling computers to understand and generate human language. Traditional NLP systems used statistical methods, but modern systems rely heavily on transformer architectures. The Transformer model was introduced in 2017 by Vaswani and colleagues in a paper titled “Attention Is All You Need.” This architecture replaced recurrent networks with attention mechanisms and enabled much larger language models to be trained efficiently.

BERT, which stands for Bidirectional Encoder Representations from Transformers, was introduced by Google in 2018. It was designed to understand context by reading text in both directions simultaneously. BERT significantly improved performance on many NLP benchmarks, including question answering and sentence classification. Later models such as RoBERTa and DistilBERT were developed to improve efficiency and performance.

In recent years, large language models have scaled dramatically. GPT-3 was released by OpenAI in 2020 and contains 175 billion parameters. GPT-4 followed later with further improvements in reasoning and multimodal capabilities. These models are trained on massive datasets and require significant computational resources, often using thousands of GPUs during training."""

question = """What architecture replaced recurrent networks?

"""

print(qa(question=question, context=context))

{'score': 0.49280398711562157, 'start': 1859, 'end': 1879, 'answer': 'attention mechanisms'}
